In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import torch 
import torch.nn as nn
import yfinance as yf
import keras 
import os
import tensorflow
import datetime 
import requests
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [3]:
#takes any ticker and returns X as past # days and Y as tmr's price, split into train and test
def DataPrep(ticker='aapl',daysago=100,split=0.9):
    df = yf.Ticker(ticker.upper()).history(period='max')
    if df.shape[1] == 7:
        df = df.drop(columns=['Dividends','Stock Splits'])
    df = df.drop(columns=['Open','High','Low','Volume'])
    for i in range(daysago,-2,-1):
        df['{}daysago'.format(i)] = df['Close'].shift(i)
        df = df.copy()
    df = df.drop(columns=['Close'])
    df = df.dropna()
    sc = MinMaxScaler(feature_range=(0,1))
    df_scaled = sc.fit_transform(df)
    X,Y = df_scaled[:,:daysago+1],df_scaled[:,-1]
    split = int(len(X)*split)
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
    X_train, X_test = X_train.reshape((-1,daysago+1,1)), X_test.reshape((-1,daysago+1,1))
    Y_train, Y_test = Y_train.reshape((-1,1)), Y_test.reshape((-1,1))
    return X_train, X_test, Y_train, Y_test

#takes any list of tickers, compiles x_train, x_test, y_train, y_test, @ a set daysago and split
def CompileData(tickers=['aapl','nvda'],daysago=100,split=0.9):
    X_train,X_test,Y_train,Y_test = [],[],[],[]
    X_train,X_test,Y_train,Y_test = np.array(X_train).reshape(-1,daysago+1,1),np.array(X_test).reshape(-1,daysago+1,1), np.array(Y_train).reshape(-1,1), np.array(Y_test).reshape(-1,1)
    for ticker in tickers:
        dataprep = DataPrep(ticker,daysago,split)
        X_train = np.append(X_train,dataprep[0],axis=0)
        X_test = np.append(X_test,dataprep[1],axis=0)
        Y_train = np.append(Y_train,dataprep[2],axis=0)
        Y_test = np.append(Y_test,dataprep[3],axis=0)
    return X_train, X_test, Y_train, Y_test
CompileData()

(array([[[3.09576887e-04],
         [2.83417173e-04],
         [2.48533543e-04],
         ...,
         [2.51314495e-04],
         [2.40308628e-04],
         [2.45811498e-04]],
 
        [[2.83417173e-04],
         [2.48533543e-04],
         [2.59433515e-04],
         ...,
         [2.40308628e-04],
         [2.45811498e-04],
         [2.49480110e-04]],
 
        [[2.48533543e-04],
         [2.59433515e-04],
         [2.72517276e-04],
         ...,
         [2.45811498e-04],
         [2.49480110e-04],
         [2.40308628e-04]],
 
        ...,
 
        [[2.44151403e-01],
         [2.40928764e-01],
         [2.41532389e-01],
         ...,
         [1.84420450e-01],
         [1.77665557e-01],
         [1.71949296e-01]],
 
        [[2.40928764e-01],
         [2.41532389e-01],
         [2.41047347e-01],
         ...,
         [1.77665557e-01],
         [1.71949296e-01],
         [1.71934550e-01]],
 
        [[2.41532389e-01],
         [2.41047347e-01],
         [2.45865132e-01],
         

In [4]:
def ReadIndex(filename='SP500',blacklist=['BRK.B','BF.B']):
    df = pd.read_csv('{}.csv'.format(filename))
    ListOfTickers = df['Symbol'].tolist()
    for ticker in blacklist:
        ListOfTickers.remove(ticker)
    return ListOfTickers

In [5]:
# function to get the CIK of a company from the SEC database
def get_cik(ticker):
    ticker_symbol = ticker.upper()

    # getting all the companies ticker and CIK from the SEC database as a dictionary
    companyTickers = requests.get('https://www.sec.gov/files/company_tickers.json', headers={'User-Agent': "testing@gmail.com"}).json().values()

    # returns {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'} for each company
    for company in companyTickers:
        if company['ticker'] == ticker_symbol:
            return(company)

    return('Ticker CIK not found in SEC database')

# function to get the SEC financial reports of a company
def sec_filings(ticker,metrics=['EarningsPerShareBasic']):

    ticker_symbol = ticker.upper()

    # grab CIK and fill leading zeroes
    cik = str(get_cik(ticker_symbol)['cik_str']).zfill(10)

    filingData = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',headers={'User-Agent': "testing@gmail.com"}).json()

    # used to check the key names for category of data
    # print(filingData['facts']['us-gaap'][f'{metrics[0]}']['units']['USD/shares'])

    # this is how to grab the data from the json api
    metricList = []
    for metric in metrics:
        valList = []
        for i in filingData['facts']['us-gaap'][f'{metric}']['units']['USD/shares']:
            val = i['val']
            valList.append(val)
        metricList.append(valList)

    return metricList
sec_filings('aos')

[[1.53,
  1.38,
  0.85,
  2.7,
  1.15,
  2.37,
  2.37,
  0.68,
  0.68,
  1.56,
  1.04,
  0.54,
  0.36,
  0.36,
  2.61,
  1.74,
  1.05,
  0.7,
  0.7,
  2.44,
  2.44,
  2.44,
  0.7,
  0.89,
  0.89,
  0.89,
  0.89,
  1.98,
  1.98,
  1.09,
  1.09,
  1.09,
  1.09,
  5.93,
  5.93,
  3.94,
  3.94,
  3.94,
  3.94,
  6.63,
  6.63,
  3.31,
  0.69,
  0.69,
  1.03,
  1.03,
  0.52,
  0.51,
  1.79,
  0.9,
  0.76,
  0.76,
  0.38,
  0.38,
  2.59,
  1.3,
  0.8,
  0.8,
  0.4,
  0.4,
  3.44,
  1.72,
  1.72,
  0.85,
  0.42,
  0.42,
  0.42,
  0.42,
  0.42,
  0.88,
  0.88,
  0.45,
  0.45,
  0.45,
  0.45,
  1.38,
  1.38,
  0.5,
  0.5,
  0.5,
  0.5,
  1.84,
  1.84,
  1.84,
  0.46,
  0.46,
  0.51,
  0.51,
  0.51,
  0.51,
  1.15,
  1.15,
  0.63,
  0.63,
  0.63,
  0.63,
  1.71,
  1.71,
  0.56,
  0.56,
  0.56,
  0.56,
  2.3,
  2.3,
  1.15,
  0.59,
  0.59,
  0.65,
  0.65,
  0.33,
  0.65,
  1.45,
  1.45,
  0.8,
  0.8,
  0.8,
  0.4,
  2.28,
  1.14,
  0.83,
  0.83,
  0.41,
  0.42,
  3.19,
  1.59,
  1.59,
  0.91,
  0.

In [6]:
for ticker in ReadIndex():
    print(ticker)
    e = sec_filings(ticker)


MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB


KeyError: 'EarningsPerShareBasic'